In [17]:
from flask import Flask
import datetime
from flask import request # used to parse payload
from youtube_transcript_api import YouTubeTranscriptApi
from youtube_transcript_api.formatters import TextFormatter
from flask import render_template
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from summarizer import Summarizer
from flask import abort
from flask_cors import CORS

# define a variable to hold you app
app = Flask(__name__)
CORS(app)

@app.route('/')
def hello():
    return render_template('index.html')

@app.route('/time', methods=['GET'])
def get_time():
    return str(datetime.datetime.now())

@app.route('/api/summarize', methods=['GET'])
def GetUrl():
    """
    Called as /api/summarize?youtube_url='url'
    """
    # if user sends payload to variable name, get it. Else empty string
    video_url = request.args.get('youtube_url', '') 
    # if(len(video_url) == 0) or (not '=' in video_url):
    #   print("f")
    #   abort(404)
    
    video_id = video_url.split('=')[1]
    response = GetTranscript(video_id)
    return response

def abs_sum(text, model, tokenizer):

    tokens_input = tokenizer.encode("summarize: "+text, return_tensors='pt',
                                    max_length=tokenizer.model_max_length,
                                    truncation=True)

    summary_ids = model.generate(tokens_input, min_length=80, max_length=150,
                                length_penalty=15, num_beams=4)

    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    return summary

def Summarize(text):
    model = AutoModelForSeq2SeqLM.from_pretrained('t5-base')
    tokenizer = AutoTokenizer.from_pretrained('t5-base')
    bert_model = Summarizer()
    ext_summary = bert_model(text, max_length=400)

    summary_2 = abs_sum(ext_summary, model, tokenizer)
    return summary_2

# video id are the last characters in the link of youtube video
def GetTranscript(video_id):
    transcript = YouTubeTranscriptApi.get_transcript(video_id)
    formatter = TextFormatter()
    text_formatted = formatter.format_transcript(transcript)
    f = open('transcript.txt', 'w', encoding='utf-8')
    f.write(text_formatted)
    # print(text_formatted)
    # return text_formatted
    return Summarize(text_formatted)

# server the app when this file is run
if __name__ == '__main__': 
  app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [11/May/2021 06:09:14] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/May/2021 06:09:15] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [11/May/2021 06:09:16] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/May/2021 06:11:56] "GET /api/summarize?youtube_url=https://www.youtube.com/watch?v=iXNsAYOTzgM HTTP/1.1" 200 -
127.0.0.1 - - [11/May/2021 06:12:46] "GET /api/summarize?youtube_url=https://www.youtube.com/watch?v=iXNsAYOTzgM HTTP/1.1" 200 -
127.0.0.1 - - [11/May/2021 06:12:54] "GET /api/summarize?youtube_url=https://www.youtube.com/watch?v=iXNsAYOTzgM HTTP/1.1" 200 -
127.0.0.1 - - [11/May/2021 06:13:13] "GET /api/summarize?youtube_url=https://www.youtube.com/watch?v=iXNsAYOTzgM HTTP/1.1" 200 -
127.0.0.1 - - [11/May/2021 06:13:51] "GET /api/summarize?youtube_url=https://www.youtube.com/watch?v=iXNsAYOTzgM HTTP/1.1" 200 -
127.0.0.1 - - [11/May/2021 06:14:26] "GET /api/summarize?youtube_url=https://www.youtube.com/watch?v=iXN

In [14]:
! ngrok authtoken 1sLdQwHD2kh1srvZBTeUlt2xPc2_2DSPydDnqQaoMLLz3xqVx

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [2]:
from pyngrok import ngrok
ngrok.kill()

In [16]:
ngrok.connect(5000)

<NgrokTunnel: "http://915557d30e19.ngrok.io" -> "http://localhost:5000">

In [3]:
!pip install pyngrok

     |████████████████████████████████| 747kB 6.8MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.0.5-cp37-none-any.whl size=19246 sha256=a647d443a1a7c6cb6587c09fbe55d1b457ffe88992281c12ef02b21f48d1720c
  Stored in directory: /root/.cache/pip/wheels/0c/13/64/5ebbcc22eaf53fdf5766b397c1fb17c83f5775fdccf0ea1b88
Successfully built pyngrok


In [2]:
!pip install -r requirements.txt

     |████████████████████████████████| 9.8MB 6.3MB/s 
     |████████████████████████████████| 184kB 52.9MB/s 
     |████████████████████████████████| 3.2MB 32.7MB/s 
     |████████████████████████████████| 184kB 45.9MB/s 
     |████████████████████████████████| 634kB 33.8MB/s 
     |████████████████████████████████| 10.3MB 29.2MB/s 
     |████████████████████████████████| 15.3MB 149kB/s 
     |████████████████████████████████| 51kB 5.5MB/s 
     |████████████████████████████████| 3.0MB 23.5MB/s 
     |████████████████████████████████| 9.1MB 17.5MB/s 
     |████████████████████████████████| 61kB 7.4MB/s 
     |████████████████████████████████| 727kB 40.2MB/s 
     |████████████████████████████████| 61kB 7.4MB/s 
     |████████████████████████████████| 901kB 29.0MB/s 
     |████████████████████████████████| 22.3MB 1.4MB/s 
     |████████████████████████████████| 27.4MB 98kB/s 
     |████████████████████████████████| 1.2MB 33.9MB/s 
     |████████████████████████████████| 122kB 54.9MB/s 